##### This script is for testing the trained dynamic model on completely independent dataset on the laptop. Since there are only restricted resourses on the laptop, i don't want to make any preprocessing of the videos for the whole dataset. Therefor I define the generator that produces data sample on demand from a randomly chosen video right before feeding it into the model.

In [1]:
#%%writefile create_tf_dataset.py
import pandas as pd
import numpy as np
import cv2
import datetime
import os
from random import shuffle,sample
import math
from tqdm import tqdm
import re
import timeit
import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
%matplotlib inline

##### Defining the parameters path to test dataset, sequence length seq_len and batch_size. 

In [ ]:
working_directory = '../new_data/'
seq_len=6
batch_size=1

##### This cell constructs the list of all videos (the file pathes) and also list of all csv-files with numerical data in the given directory. 

In [2]:
video_names = [root+'/'+file for root, _, files in os.walk(working_directory) for file in files if file.endswith('flv')]
csv_names   = [root+'/'+file for root, _, files in os.walk(working_directory) for file in files if file.endswith('csv')]

#### Creating one dataframe with numerical values stacking all the csv files together in a panda dataframe

In [3]:
wind_data=pd.DataFrame()
for path in csv_names:
    tabel=pd.read_csv(path,parse_dates=False)#.reset_index(drop=True)
    wind_data=pd.concat([wind_data,tabel]).reset_index(drop=True)

##### Resulting dataframe looks like this

In [4]:
wind_data.tail()

,date,Avg. wind,wind gusts
708,2018-5-31-16-7,15,20
709,2018-5-31-16-12,16,19
710,2018-5-31-16-17,13,17
711,2018-5-31-16-22,12,17
712,2018-5-31-16-27,13,16


##### Function to get time stamp from the video name

In [5]:
def vn_to_time(vn):
    return datetime.datetime.strptime(vn.split('/')[-1].split('.')[0], '%Y-%m-%d-%H-%M')

##### testing the function

In [6]:
vn_to_time(video_names[0])

datetime.datetime(2018, 5, 26, 11, 58)

##### Function that produces the numerical values for given video. It finds two closest in time data points and make interpolation. It also takes care of some edge cases.

In [7]:
def wind_for_video(vn,wind_data=wind_data):
    vid_time=vn_to_time(vn) #get the timestame for the given video
    
    wind_data["time_diff"]=wind_data['date'].apply(lambda x: #get the difference between time stamps of video and all datapoints
                                                   (datetime.datetime.strptime(x,'%Y-%m-%d-%H-%M')-vid_time).total_seconds())
    wind_data['time_dif_abs']=wind_data['time_diff'].apply(abs) #get absolute values of this difference
    wd=wind_data.sort_values(['time_dif_abs']).head(2).reset_index(drop=True) #get two closest datapoints
    
    if wd.empty:  
        return None
    if wd['time_dif_abs'].min()>500: # edge case when there is no datapoints closer than 500sec. 
                                     # No data will be produced for such a video
        return None
    
    elif wd['time_dif_abs'].max()>1000: # egde case when only one data point is close to the given video
        awind=wd['Avg. wind'].loc[wd['time_dif_abs'].idxmin()]
        gwind=wd['wind gusts'].loc[wd['time_dif_abs'].idxmin()]
    
    dt = wd['time_diff'].loc[1]-wd['time_diff'].loc[0]
    if abs(dt)<1:  # Edge case when two closest datapoints almost coinside 
        awind=(wd['Avg. wind'].loc[0]+wd['Avg. wind'].loc[1])/2
        gwind=(wd['wind gusts'].loc[0]+wd['wind gusts'].loc[1])/2
    else: # a regular case when there are two points to interpolate
        awind=(wd['Avg. wind'].loc[0]*wd['time_diff'].loc[1] - wd['Avg. wind'].loc[1]*wd['time_diff'].loc[0])/dt
        gwind=(wd['wind gusts'].loc[0]*wd['time_diff'].loc[1] - wd['wind gusts'].loc[1]*wd['time_diff'].loc[0])/dt
    return [awind,gwind]
    

##### testing the function for some specific video file from the list

In [8]:
wind_for_video(video_names[100])

[11.0, 16.0]

##### Create a new dataframe with three columns - video path, and two corresponding numerical values as calculated using function abow.

In [9]:
df_wind=pd.DataFrame()
for video in video_names:
    num_values=wind_for_video(video) 
    if num_values is None: #if there is no numerical value for the given video available, skip this video
        continue
    df=pd.DataFrame([[video]+num_values],columns=['video','av.wn.','wn.gs.'])
    df_wind=df_wind.append(df)
df_wind.reset_index(drop=True,inplace=True)

##### Using this dataframe makes it easy to select any subset of numerical values from the test dataset. For exapmple, we can take only samples where average wind less than 5knt. In this way we can check the performance of the model on any specific subset

In [10]:
df_wind[df_wind['av.wn.']<5].reset_index(drop=True).tail()

,video,av.wn.,wn.gs.
52,../new_data/new-tangana/2018-05-31-10-18.flv,4.0,5.8
53,../new_data/new-tangana/2018-05-31-10-24.flv,2.0,6.2
54,../new_data/new-tangana/2018-05-31-10-29.flv,3.8,7.6
55,../new_data/new-tangana/2018-05-31-10-35.flv,3.2,7.0
56,../new_data/new-tangana/2018-05-31-10-40.flv,1.6,5.8


##### Function that read frames from the given video and returns sequence of frames as a numpy array. The fisrt frame is taken randomly between frame 0 and 300.

In [11]:
def read_frames(videoPath,seq_len):
    count = 0
    vidcap = cv2.VideoCapture(videoPath)
    offset=np.random.randint(0,high=300) #generates random first frame
    seq=[]
    success = True
    while success:
        success,image = vidcap.read()
        if not success:
            return None
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if count >= offset:
            seq.append(image)
            if count >= offset+seq_len-1:
                return np.stack(seq)[:,:448,:704,:]
        count = count + 1
    

##### The generator that produces batch of random samples of data. It returns a tuple of 1) video frame sequence in numpy format and 2)numerical data. This generator can then be used with the method "batch_predict_generator" for keras model

In [12]:
def batch_gen(seq_len,data_frame=df_wind,batch_size=1):
    batch_frames=[]
    batch_data=[]
    while True:        
        df=list(data_frame.sample().values[0]) #sample the dataframe and create list from its values
        seq=read_frames(df[0],seq_len) #read sequence of frames for the given video
        num_dat=np.array([df[1],df[2]]) # get the numerical data for the video
        if df[1] is None or df[2] is None:
            continue
        if seq is None:
            continue
        batch_frames.append(seq/255.)
        batch_data.append(num_dat)
        if len(batch_frames)>=batch_size:
            yield np.stack(batch_frames),np.stack(batch_data) 
            batch_frames=[]
            batch_data=[]   

##### testing the generator

In [13]:
gen=batch_gen(seq_len)

In [14]:
next(gen)[0].shape

(1, 6, 448, 704, 3)

##### Loading the model to test

In [15]:
import tensorflow as tf
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
from tqdm import tqdm
import keras as keras
from keras.applications import MobileNetV2
from keras.models import Model,load_model,Sequential
from keras.layers import *
import keras.backend as K

Using TensorFlow backend.


##### Loading the stationary model by parts. The first convolutional base.

In [16]:
#this is convolutional base
conv_base=MobileNetV2(include_top=False)#,input_tensor=next_element[0]

C:\Users\ttnnn\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:310: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


##### Adding self attention on top of convolutional base 

In [17]:
inp=Input((None,None,None))
x = inp

x=conv_base(x)

# next 5 layers are attention mechanism

weights=Conv2D(1,(3,3),activation='sigmoid',padding='SAME',kernel_initializer=keras.initializers.Zeros(),
                                               bias_initializer=keras.initializers.Ones())(x)

#next two lines make the weights sum up to 1: weights->weights/sum(weights)
norm = Lambda(lambda t: 1/K.sum(t,axis=[1,2]))(weights)

weights = merge.multiply([norm,weights])

#next two lines calculate weighted average
x = merge.multiply([x,weights])

x = Lambda(lambda t: K.sum(t,axis=[1,2]))(x)

##### Defining the conv_base + self attention  part of the model

In [18]:
model=Model(inputs=[inp],outputs=[x])

##### Loading the top layers of the stationary model

In [19]:
mobinet1=load_model('../Models/mobile_mini_top_attention_3.h5',custom_objects={'relu6':ReLU(6.),'tf':tf},compile=False)

##### Redefining the loaded model to have the same architecture as the part defined previosely. In this case the trained weights can be assined. Unfurtunately loaded model does not work with TimeDistributed wrapper, because of the bug. That's why it was necessary to define the model and then load the weights instead of just working with loaded model.

In [20]:
mobinet2=Model(inputs=[mobinet1.layers[0].input],outputs=[mobinet1.layers[6].output])

##### Assigning the weights

In [21]:
model.set_weights(mobinet2.get_weights())

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, N 0                                            
__________________________________________________________________________________________________
mobilenetv2_1.00_224 (Model)    (None, None, None, 1 2257984     input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 11521       mobilenetv2_1.00_224[1][0]       
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
multiply_1

##### As it can be seen from summary the model produces a feature vector of 1280 numbers for a given frame. Now we need to apply this feature extractor to a sequence of the frames by wrapping the feature extractor with TimeDistributed wrapper.

In [23]:
mob_time=TimeDistributed(model)

inp=Input((None,None,None,None))

x=inp
x=mob_time(x)

In [24]:
time_mob=Model(inputs=[inp], outputs=[x])

##### Adding the final layers on the top that include sequence analyser. It can be 1D convolition along the time axis or any type of RNN layer (simple RNN, LSTM, GRU etc). There is also residual connection of simple averaging in time, thus the performance of the stationary model is not lost.

In [25]:
dense1=mobinet1.layers[7] # this is top layer in stationary model

In [26]:
inp0 = Input(shape=(seq_len,1280))# 
inp1 = Input(shape=(2,))
inp2 = Input(shape=(2,))

x = inp0
labels = inp1
loss_weights = inp2

avg = TimeDistributed(dense1)(x) # top layer of stationary model applyed on sequence of frames 

avg = GlobalAveragePooling1D()(avg) # taking the average of the resulting prediction along time axis

gru=GRU(2)(x) # sequence analyser

prediction = Add()([avg,gru]) #combining the both predictions by adding them up

In [27]:
model1=Model(inputs=[inp0], outputs=[prediction])

In [28]:
model_trained=load_model('../Models/mobile_dynamics_gru_3.h5',custom_objects={'relu6':ReLU(6.),'tf':tf},compile=False)

In [29]:
model_trained=Model(model_trained.inputs[0],model_trained.layers[-7].output)

In [30]:
model1.set_weights(model_trained.get_weights())

In [31]:
final_model = Sequential([time_mob,model1])


In [32]:
adam = keras.optimizers.Adam(lr=.0001)

final_model.compile(optimizer=adam,loss='mse')

In [33]:
df_wind[df_wind['av.wn.']<30].describe()

,av.wn.,wn.gs.
count,513.000000,513.000000
mean,10.201040,13.470565
std,3.623536,3.534618
min,0.000000,4.000000
25%,9.000000,11.600000
50%,11.000000,14.200000
75%,12.600000,16.000000
max,16.800000,19.600000


In [34]:
df_wind[ (df_wind['av.wn.']<10)].describe()#(df_wind['av.wn.']<15) &

,av.wn.,wn.gs.
count,156.000000,156.000000
mean,5.730342,9.257692
std,2.932779,2.399033
min,0.000000,4.000000
25%,2.800000,7.000000
50%,6.000000,9.600000
75%,8.700000,11.000000
max,9.800000,15.000000


In [35]:
gen=batch_gen(seq_len,data_frame=df_wind[df_wind['av.wn.']<30])

In [36]:
corr=0
incorr=0
for i in range(200):
    inp=next(gen)
    if inp[1] is None:
        continue
    pred=final_model.predict_on_batch(inp[0])
    true_val=inp[1]
    err=np.sqrt(np.mean(np.square(pred-true_val)))
    if err<5.:
        corr+=1
    else:
        incorr+=1
    print (pred,true_val)
    print (err)
    print ('Correct:', corr, 'Incorrect:', incorr)
    
        
    

[[14.062221 20.353931]] [[11. 14.]]
4.98746626150755
Correct: 1 Incorrect: 0
[[13.081132 21.997755]] [[12. 16.]]
4.309403201653519
Correct: 2 Incorrect: 0
[[11.874195 16.375935]] [[11.6 12.6]]
2.677019337060712
Correct: 3 Incorrect: 0
[[14.278511 22.76069 ]] [[12.8 15.2]]
5.447478252054837
Correct: 3 Incorrect: 1
[[13.682765 19.855743]] [[14.8 19. ]]
0.9951157747278105
Correct: 4 Incorrect: 1
[[13.237   20.46991]] [[13. 18.]]
1.7545117538110184
Correct: 5 Incorrect: 1
[[12.481547 19.101498]] [[13. 16.]]
2.2235198243208614
Correct: 6 Incorrect: 1
[[11.560575 16.188858]] [[13. 17.]]
1.168310098195459
Correct: 7 Incorrect: 1
[[15.434274 23.161102]] [[11. 13.]]
7.83934892927549
Correct: 7 Incorrect: 2
[[10.690453 16.593449]] [[10.8 14.4]]
1.552935537898608
Correct: 8 Incorrect: 2
[[11.894026 20.577429]] [[ 9.6 11.6]]
6.551976135518263
Correct: 8 Incorrect: 3
[[14.653056 22.589943]] [[10. 13.]]
7.5371724448679425
Correct: 8 Incorrect: 4
[[15.6993885 22.503513 ]] [[13.2 17.8]]
3.766296629164

In [37]:
gen=batch_gen(seq_len,data_frame=df_wind[df_wind['av.wn.']<10])

corr=0
incorr=0
for i in range(200):
    inp=next(gen)
    if inp[1] is None:
        continue
    pred=final_model.predict_on_batch(inp[0])
    true_val=inp[1]
    err=np.sqrt(np.mean(np.square(pred-true_val)))
    if err<5.:
        corr+=1
    else:
        incorr+=1
    print (pred,true_val)
    print (err)
    print ('Correct:', corr, 'Incorrect:', incorr)
    
        
    

[[ 8.687351 12.635367]] [[3.2 7. ]]
5.561851721593618
Correct: 0 Incorrect: 1
[[ 6.081357 10.489956]] [[2.8 6.8]]
3.491638464654476
Correct: 1 Incorrect: 1
[[11.737333 17.040716]] [[1.6 5.2]]
11.02197999085698
Correct: 1 Incorrect: 2
[[ 7.026161 12.143874]] [[2.8 6.8]]
4.817542400645471
Correct: 2 Incorrect: 2
[[ 8.768993 14.963949]] [[5. 7.]]
6.230160431193799
Correct: 2 Incorrect: 3
[[ 8.846273 11.974252]] [[ 8. 10.]]
1.5188562582158704
Correct: 3 Incorrect: 3
[[10.080208 15.884628]] [[ 0. 11.]]
7.92054869874482
Correct: 3 Incorrect: 4
[[10.69988 17.11053]] [[ 9.2 11. ]]
4.449057555380215
Correct: 4 Incorrect: 4
[[ 9.744469 14.681075]] [[6.4 7.4]]
5.665665246330572
Correct: 4 Incorrect: 5
[[ 9.800627 15.603034]] [[ 9.8 11. ]]
3.254836599371899
Correct: 5 Incorrect: 5
[[12.5439005 17.629168 ]] [[ 8.6 10.8]]
5.5763733999724705
Correct: 5 Incorrect: 6
[[ 8.461794 13.411403]] [[2.4 8.8]]
5.3856466725697905
Correct: 5 Incorrect: 7
[[ 9.408236 14.535882]] [[2.6 6.6]]
7.3935882479409205
Cor